#How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

##How to wrap Keras models for use in scikit-learn and how to use grid search.

##How to grid search common neural network parameters, such as learning rate, dropout rate, epochs, and number of neurons.

##How to define your own hyperparameter tuning experiments on your own projects.

How to use Keras models in scikit-learn

How to use grid search in scikit-learn

How to tune batch size and training epochs

How to tune optimization algorithms

How to tune learning rate and momentum

How to tune network weight initialization

How to tune activation functions

How to tune dropout regularization

How to tune the number of neurons in the hidden layer


#How to Use Keras Models in scikit-learn
Keras models can be used in scikit-learn by wrapping them with the KerasClassifier or KerasRegressor class from the module SciKeras. You may need to run the command pip install scikeras first to install the module.

To use these wrappers, you must define a function that creates and returns your Keras sequential model, then pass this function to the model argument when constructing the KerasClassifier class.

For example:

def create_model():

 ...

 return model

##model = KerasClassifier(model=create_model)
*************************************************************************

The constructor for the KerasClassifier class can take default arguments that are passed on to the calls to model.fit(), such as the number of epochs and the batch size.

For example:

def create_model():

 ...

 return model

##model = KerasClassifier(model=create_model, epochs=10)
************************************************************

The constructor for the KerasClassifier class can also take new arguments that can be passed to your custom create_model() function. These new arguments must also be defined in the signature of your create_model() function with default parameters.

For example:

def create_model(dropout_rate=0.0):

 ...

 return model

##model = KerasClassifier(model=create_model, dropout_rate=0.2)

#How to Use Grid Search in scikit-learn
Grid search is a model hyperparameter optimization technique.

In scikit-learn, this technique is provided in the GridSearchCV class.

When constructing this class, you must provide a dictionary of hyperparameters to evaluate in the param_grid argument. This is a map of the model parameter name and an array of values to try.

By default, accuracy is the score that is optimized, but other scores can be specified in the score argument of the GridSearchCV constructor.

By default, the grid search will only use **one thread**. By setting the **n_jobs** argument in the GridSearchCV constructor to -1, the process will **use all cores** on your machine. However, sometimes this may interfere with the main neural network training process.

The GridSearchCV process will then **construct and evaluate one model for each combination of parameters.** Cross validation is used to evaluate each individual model, and the **default of 3-fold cross validation** is used, although you can override this by specifying the cv argument to the GridSearchCV constructor.

Below is an example of defining a simple grid search:

param_grid = dict(epochs=[10,20,30])

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X, Y)

Once completed, you can access the outcome of the grid search in the result object returned from grid.fit(). The **best_score_** member provides access to the best score observed during the optimization procedure, and the **best_params_** describes the combination of parameters that achieved the best results.


***********************************************************************
#Problem Description

Now that you know how to use Keras models with scikit-learn and how to use grid search in scikit-learn, let’s look at a bunch of examples.

All examples will be demonstrated on a small standard machine learning dataset called the Pima Indians onset of diabetes classification dataset. This is a small dataset with all numerical attributes that is easy to work with.

Download the dataset and place it in your currently working directly with the name pima-indians-diabetes.csv (update: download https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv).

***************************************************************************
#Note on Parallelizing Grid Search
All examples are configured to use parallelism (n_jobs=-1).

If you get an error like the one below:

INFO (theano.gof.compilelock): Waiting for existing lock by process '55614' (I am process '55613')
INFO (theano.gof.compilelock): To manually release the lock, delete ...
Kill the process and change the code to not perform the grid search in parallel; set n_jobs=1.
*************************************************************************
#How to Tune Batch Size and Number of Epochs
In this first simple example, you will look at tuning the batch size and number of epochs used when fitting the network.

The **batch size** in iterative gradient descent is **the number of patterns shown to the network before the weights are updated.** It is also an optimization in the training of the network, defining how many patterns to read at a time and keep in memory.

The **number of epochs** is the **number of times the entire training dataset is shown to the network during training.**

###Some networks are sensitive to the batch size, such as LSTM recurrent neural networks and Convolutional Neural Networks.

Here you will evaluate a suite of different mini-batch sizes from 10 to 100 in steps of 20.
************************************************************************



In [1]:
!pip install scikeras

In [2]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2023-12-03 05:36:01--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.002s  

2023-12-03 05:36:01 (10.2 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



#Note: Your results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average outcome.

In [3]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier


# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)


# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")


# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

print(X)
print(Y)
print(X.shape)
print(Y.shape)

# create model
model = KerasClassifier(model=create_model, verbose=1)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 

##You can see that the batch size of 10 and 100 epochs achieved the best result of about 70% accuracy.

#How to Tune the Training Optimization Algorithm
Keras offers a suite of different state-of-the-art optimization algorithms.

In this example, you will tune the optimization algorithm used to train the network, each with default parameters.

This is an odd example because often, you will choose one approach a priori and instead focus on tuning its parameters on your problem.

Here, you will evaluate the suite of optimization algorithms supported by the Keras API.

In [5]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier


# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# return model without compile
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)


# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]


# create model
model = KerasClassifier(model=create_model, loss="binary_crossentropy", epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)


# summarize results
print("**********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 30.0094
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 5.0262
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 2.7248
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 2.4871
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 2.2396
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 2.0263
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 1.8492
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 1.7171
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 1.5101
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 1.4047
Epoch 11/100
77/77 [==============================] - 0s 2ms/step - loss: 1.3030
Epoch 12/100
77/77 [==============================] - 0s 2ms/step - loss: 1.1738
Epoch 13/100
77/77 [================

#The KerasClassifier wrapper will not compile your model again if the model is already compiled. Hence the other way to run GridSearchCV is to set the optimizer as an argument to the create_model() function, which returns an appropriately compiled model like the following:

In [7]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):                  #<-----------Taken optimizer as Adam
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(model__optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)


# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 3ms/step - loss: 5.3955 - accuracy: 0.5768
Epoch 2/100
77/77 [==============================] - 0s 5ms/step - loss: 2.2352 - accuracy: 0.6497
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 1.7388 - accuracy: 0.6445
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 1.5266 - accuracy: 0.6237
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 1.4000 - accuracy: 0.6107
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 1.3578 - accuracy: 0.6211
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 1.2555 - accuracy: 0.6302
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 1.2451 - accuracy: 0.6224
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 1.1399 - accuracy: 0.6458
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 1.1485 - accuracy: 0.6367
Epoch 11/

##Note that in the above, you have the **prefix model__** in the parameter dictionary param_grid. This is required for the KerasClassifier in the SciKeras module to make clear that the parameter needs to route into the create_model() function as arguments, rather than some parameter to set up in compile() or fit(). See also the routed parameter section of SciKeras documentation.

#The results suggest that the ADAM optimization algorithm is the best with a score of about 70% accuracy.



#How to Tune Learning Rate and Momentum
It is common to pre-select an optimization algorithm to train your network and tune its parameters.

By far, the most common optimization algorithm is **plain old Stochastic Gradient Descent (SGD)** because it is so well understood. In this example, you will look at optimizing the **SGD learning rate and momentum parameters.**

The **learning rate** controls how much to update the weight at the end of each batch, and the **momentum** controls how much to let the previous update influence the current weight update.

You will try a suite of small standard learning rates and momentum values from 0.2 to 0.8 in steps of 0.2, as well as 0.9 (because it can be a popular value in practice). In Keras, the way to set the learning rate and momentum is the following:

...

###optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.2)

In the SciKeras wrapper, you will route the parameters to the optimizer with the **prefix optimizer__.**

Generally, it is a good idea to also include the number of epochs in an optimization like this as there is a dependency between the amount of learning per batch (learning rate), the number of updates per epoch (batch size), and the number of epochs

In [8]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier


# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, loss="binary_crossentropy", optimizer="SGD", epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 3ms/step - loss: 2.3505
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.7006
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6644
Epoch 4/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6526
Epoch 5/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6493
Epoch 6/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6343
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6369
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6386
Epoch 9/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6285
Epoch 10/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6220
Epoch 11/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6235
Epoch 12/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6162
Epoch 13/100
77/77 [=================

##You can see that SGD is not very good on this problem; nevertheless, the best results were achieved using a learning rate of 0.001 and a momentum of 0.8 with an accuracy of about 68%.

#How to Tune Network Weight Initialization
Neural network weight initialization used to be simple: use small random values.

Now there is a suite of different techniques to choose from. Keras provides a laundry list.

In this example, you will look at tuning the selection of network weight initialization by evaluating all the available techniques.

You will use the same weight initialization method on each layer.

 Ideally, it may be better to use different weight initialization schemes according to the activation function used on each layer.

 In the example below, you will use a rectifier for the hidden layer. Use sigmoid for the output layer because the predictions are binary. The weight initialization is now an argument to create_model() function, where you need to use the model__ prefix to ask the KerasClassifier to route the parameter to the model creation function.

In [9]:
# Use scikit-learn to grid search the weight initialization
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
# Function to create model, required for KerasClassifier

def create_model(init_mode='uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']   #initialization mode
param_grid = dict(model__init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 0.6598 - accuracy: 0.6471
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6479 - accuracy: 0.6484
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6320 - accuracy: 0.6628
Epoch 4/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6321 - accuracy: 0.6615
Epoch 5/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6271 - accuracy: 0.6693
Epoch 6/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6203 - accuracy: 0.6667
Epoch 7/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6139 - accuracy: 0.6849
Epoch 8/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6170 - accuracy: 0.6849
Epoch 9/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6060 - accuracy: 0.6836
Epoch 10/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6017 - accuracy: 0.6927
Epoch 11/

##We can see that the best results were achieved with a uniform weight initialization scheme achieving a performance of about 72%.

#How to Tune the Neuron Activation Function
The activation function controls the non-linearity of individual neurons and when to fire.

Generally, the rectifier activation function is the most popular. However, it used to be the **sigmoid and the tanh functions,** and these functions may still be more suitable for different problems.

In this example, you will evaluate the suite of different activation functions available in Keras. You will only use these functions in the hidden layer, as a **sigmoid activation function is required in the output for the binary classification problem.** Similar to the previous example, this is an argument to the create_model() function, and you will use the model__ prefix for the GridSearchCV parameter grid.

Generally, it is a good idea to prepare data to the range of the different transfer functions, which you will not do in this case.

In [10]:
# Use scikit-learn to grid search the activation function
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(model__activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 0.6613 - accuracy: 0.6393
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6465 - accuracy: 0.6654
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6321 - accuracy: 0.6602
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6346 - accuracy: 0.6589
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6312 - accuracy: 0.6654
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6253 - accuracy: 0.6602
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6192 - accuracy: 0.6797
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6245 - accuracy: 0.6823
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6143 - accuracy: 0.6823
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6118 - accuracy: 0.6927
Epoch 11/

#Surprisingly (to me at least), the “linear” activation function achieved the best results with an accuracy of about 71%.

#How to Tune Dropout Regularization
In this example, you will look at tuning the **dropout rate** for regularization in an effort **to limit overfitting** and improve the model’s ability to generalize.

For the best results, dropout is best combined with a weight constraint such as the max norm constraint.

For more on using dropout in deep learning models with Keras see the post:

###Dropout Regularization in Deep Learning Models With Keras
This involves fitting both the dropout percentage and the weight constraint. We will try dropout percentages between 0.0 and 0.9 (1.0 does not make sense) and maxnorm weight constraint values between 0 and 5.

In [17]:
# Use scikit-learn to grid search the dropout rate
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from scikeras.wrappers import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):     # droprate and weight_constraint
 # create model
 model = Sequential()
 model.add(Dense(12, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(weight_constraint)))
 model.add(Dropout(dropout_rate))
 model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")
print(dataset.dtype, dataset.shape)

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)
#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

float64 (768, 9)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 0.6745 - accuracy: 0.6354
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6544 - accuracy: 0.6549
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6385 - accuracy: 0.6523
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6381 - accuracy: 0.6523
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6319 - accuracy: 0.6680
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6257 - accuracy: 0.6654
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6197 - accuracy: 0.6823
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6810
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6148 - accuracy: 0.6810
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6115 - accuracy: 0.6888
Epoch 11/

We can see that the dropout rate of 0% and the MaxNorm weight constraint of
3 resulted in the best accuracy of about 77%. You may notice some of the result is nan. Probably it is due to the issue that the input is not normalized and you may run into a degenerated model by chance.

#How to Tune the Number of Neurons in the Hidden Layer
The number of neurons in a layer is an important parameter to tune. Generally the **number of neurons** in a layer controls the representational capacity of the network, at least at that point in the topology.

Also, generally, a large enough single layer network can approximate any other neural network, at least in theory.

In this example, we will look at tuning the number of neurons in a single hidden layer. We will try values from 1 to 30 in steps of 5.

A larger network requires more training and at least the batch size and number of epochs should ideally be optimized with the number of neurons.

In [18]:
# Use scikit-learn to grid search the number of neurons
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.constraints import MaxNorm

# Function to create model, required for KerasClassifier
def create_model(neurons):             #neurons
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# load dataset
dataset = np.loadtxt("/content/pima-indians-diabetes.data.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(model__neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("***********************************************************************")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("***********************************************************************")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/100
77/77 [==============================] - 1s 2ms/step - loss: 0.6796 - accuracy: 0.6341
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6583 - accuracy: 0.6589
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6300 - accuracy: 0.6667
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6389 - accuracy: 0.6549
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6413 - accuracy: 0.6706
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6275 - accuracy: 0.6667
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6199 - accuracy: 0.6901
Epoch 8/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6369 - accuracy: 0.6719
Epoch 9/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6217 - accuracy: 0.6602
Epoch 10/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6150 - accuracy: 0.6875
Epoch 11/

#We can see that the best results were achieved with a network with 30 neurons in the hidden layer with an accuracy of about 73%.

##Tips for Hyperparameter Optimization
This section lists some handy tips to consider when tuning hyperparameters of your neural network.

**k-fold Cross Validation.** You can see that the results from the examples in this post show some variance. A default cross-validation of 3 was used, but perhaps k=5 or k=10 would be more stable. Carefully choose your cross validation configuration to ensure your results are stable.


**Review the Whole Grid.** Do not just focus on the best result, review the whole grid of results and look for trends to support configuration decisions.
Parallelize. Use all your cores if you can, neural networks are slow to train and we often want to try a lot of different parameters. Consider spinning up a lot of AWS instances.

**Use a Sample of Your Dataset.** Because networks are slow to train, try training them on a smaller sample of your training dataset, just to get an idea of general directions of parameters rather than optimal configurations.
Start with Coarse Grids. Start with coarse-grained grids and zoom into finer grained grids once you can narrow the scope.

**Do not Transfer Results.** Results are generally problem specific. Try to avoid favorite configurations on each new problem that you see. It is unlikely that optimal results you discover on one problem will transfer to your next project. Instead look for broader trends like number of layers or relationships between parameters.

Reproducibility is a Problem. Although we set the seed for the random number generator in NumPy, the results are not 100% reproducible. There is more to reproducibility when grid searching wrapped Keras models than is presented in this post.